In [ ]:
from simple.binance import getHistMonth
from simple.plotly import chartFigure, updateLines
import pandas as pd
from simple.pretty import pmap
from simple.funcs import hurst

from statsmodels.tsa.stattools import adfuller
from ipyslickgrid import show_grid
from ipywidgets import VBox

In [ ]:
frame = '1h'

tickers = [
    '1000BTTCUSDT', '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'AKROUSDT',
    'ALGOUSDT', 'ALICEUSDT', 'ALPHAUSDT', 'ANKRUSDT', 'ANTUSDT', 'API3USDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT',
    'AUDIOUSDT', 'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLZUSDT',
    'BNBBUSD', 'BNBUSDT', 'BTCBUSD', 'BTCDOMUSDT', 'BTCSTUSDT', 'BTCUSDT', 'BTSUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT',
    'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'CVCUSDT', 'DASHUSDT', 'DEFIUSDT',
    'DENTUSDT', 'DGBUSDT', 'DODOUSDT', 'DOGEBUSD', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'EGLDUSDT', 'ENJUSDT',
    'ENSUSDT', 'EOSUSDT', 'ETCUSDT', 'ETHBUSD', 'ETHUSDT', 'FILUSDT', 'FLMUSDT', 'FLOWUSDT', 'FTMUSDT', 'FTTBUSD',
    'GALAUSDT', 'GRTUSDT', 'GTCUSDT', 'HBARUSDT', 'HNTUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IMXUSDT', 'IOSTUSDT',
    'IOTAUSDT', 'IOTXUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'KSMUSDT', 'LINAUSDT', 'LINKUSDT', 'LITUSDT', 'LPTUSDT',
    'LRCUSDT', 'LTCUSDT', 'LUNAUSDT', 'MANAUSDT', 'MASKUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEARUSDT', 'NEOUSDT',
    'NKNUSDT', 'OCEANUSDT', 'OGNUSDT', 'OMGUSDT', 'ONEUSDT', 'ONTUSDT', 'PEOPLEUSDT', 'QTUMUSDT', 'RAYUSDT', 'REEFUSDT',
    'RENUSDT', 'RLCUSDT', 'ROSEUSDT', 'RSRUSDT', 'RUNEUSDT', 'RVNUSDT', 'SANDUSDT', 'SCUSDT', 'SFPUSDT', 'SKLUSDT',
    'SNXUSDT', 'SOLBUSD', 'SOLUSDT', 'SRMUSDT', 'STMXUSDT', 'STORJUSDT', 'SUSHIUSDT', 'SXPUSDT', 'THETAUSDT', 'TLMUSDT',
    'TOMOUSDT', 'TRBUSDT', 'TRXUSDT', 'UNFIUSDT', 'UNIUSDT', 'VETUSDT', 'WAVESUSDT', 'XEMUSDT', 'XLMUSDT', 'XMRUSDT',
    'XRPBUSD', 'XRPUSDT', 'XTZUSDT', 'YFIIUSDT', 'YFIUSDT', 'ZECUSDT', 'ZENUSDT', 'ZILUSDT', 'ZRXUSDT'
]

In [ ]:
def getClose(ticker: str) -> pd.Series:
    return getHistMonth('2022-02', '2022-03', ticker, frame, close_only=True, spot=False)

In [ ]:
# parallel load data from binance archive
Close = pd.concat(pmap(getClose, tickers), axis=1)
Close.head()

In [ ]:
# parallel calculate Hurst Exponent value
Hurst = pd.DataFrame(pmap(lambda t: hurst(Close[t].values), Close.columns), index=Close.columns, columns=['Hurst Exponent'])
Hurst

In [ ]:
# parallel run of Augmented Dickey-Fuller test
ADF = pd.DataFrame(pmap(adfuller, Close.dropna().values.T),
                   index=Close.columns,
                   columns=['ADF', 'P-value', 'Used Lag', 'Observations', 'Critical Values', 'Best IC'])
ADF

In [ ]:
fig = chartFigure(Close=dict(color='lightblue', width=3), height=400)

In [ ]:
def on_changed(event, grid):
    """Redraw chart with selected ticker's data"""

    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k + 1].to_dict('records')[0]
    ticker = selected['index']
    updateLines(fig, Close=Close[ticker])
    fig.layout.title.text = ticker

In [ ]:
X = Hurst.join(ADF).reset_index()
grid = show_grid(X, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False, 'rowHeight': 24, 'maxVisibleRows': 14},
                 column_options={'defaultSortAsc': False},
                 column_definitions={'Critical Values': {'width': 500}})

grid.on('selection_changed', on_changed)
grid.change_selection([0])

In [ ]:
VBox([fig, grid])